# LLM Experiments

## Setup

In [6]:
# !conda activate capstone
# !pip install langchain langchain-openai langchain-anthropic pandas

### Imports

In [7]:
!pip install openai
!pip install anthropic
!pip install langsmith
!pip install langchain
!pip install langchain-openai
!pip install langchain-anthropic
!pip install pandas
!pip install numpy
!pip install re
!pip install summarytools

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [8]:
import os
import importlib
import config
from utilities import *
from prompts import *
import pandas as pd
import numpy as np
import re
from evaluators import Evaluator
# LangChain
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager
from langchain import hub
from langsmith import evaluate
from langchain_core.output_parsers import JsonOutputParser

from summarytools import dfSummary

# Reload the config file
importlib.reload(config)

# Set the environment variables
os.environ["LANGCHAIN_API_KEY"] = config.LANGCHAIN_API_KEY
os.environ["LANGCHAIN_ENDPOINT"] = config.LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_PROJECT"] = config.LANGCHAIN_PROJECT
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY



Define evaluators

In [9]:
# Initialize evaluator
evaluator = Evaluator()

evaluators = [
    evaluator.evaluate_answer_match,
    evaluator.evaluate_llm_correct,
    evaluator.evaluate_correct_match
]

### Pandas Config

In [10]:
# Increase display options for pandas
pd.set_option('display.max_colwidth', None)  # or a large number like 1000
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

### Create Models

In [11]:
# Create a tracer
tracer = LangChainTracer(project_name=os.environ["LANGCHAIN_PROJECT"])

# Create a callback manager with the tracer
callback_manager = CallbackManager([tracer])

# Initialize some models
gpt4o = ChatOpenAI(
    model="gpt-4o",
    temperature=0.65,
    callbacks=callback_manager
)
gpt4o_mini = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.65,
    callbacks=callback_manager
)
o1_preview = ChatOpenAI(
    model="o1-preview",
    temperature=1,
    callbacks=callback_manager
)
o1_mini = ChatOpenAI(
    model="o1-mini",
    temperature=1,
    callbacks=callback_manager
)
sonnet = ChatAnthropic(
    model_name="claude-3-5-sonnet-latest",
    temperature=0.65,
    timeout=None,
    stop=None,
    callbacks=callback_manager
)
haiku = ChatAnthropic(
    model_name="claude-3-5-haiku-latest",
    temperature=0.65,
    timeout=None,
    stop=None,
    callbacks=callback_manager
)

## Data Preparation

### Load Data

In [13]:
df = pd.read_csv('../data/new/master_with_all_stats.csv')

In [7]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,answer_id[int64],Mean (sd) : 1462230.9 (80782.5)min < med < max:1321329.0 < 1462881.5 < 1603613.0IQR (CV) : 138087.8 (18.1),"198,760 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACq0lEQVR4nO3YMU8iQQCG4W9AdBGyG8TDzuZKkytIbPgBV96P9SdYWFxtcc1VdiaGQNyTsC5cc1xyyUXYWRQ+8j61w4zsO7M6YblcCth3jV0vANjE0a4XgO0KIbQlHUcOny+Xy5dtrmdbDjbUmg9MqvHQas7dkLSIHJucnZ19zbKsEzN4Op2OQwg3+xjrQYYaQmj3+/1vaZr2Yj8j9qHVmbssy1ae55+73e7PRqNRVB1fFEUi6er6+vr7+fn5tMrYyWSS3N3d9Z6eno4l7WKDvnkw7HWoNX7xtNvtDkaj0TTLslnVwTUf2nGapr3RaPRSde6Hh4fe/f39p+Fw+OPi4mJccd7V+NMkSYrBYPCr6nhJ7YgxkuofDusOhrWhOr7GiqJIyrK8Ojk5uY18YCrLMpWUhhCqDk0Xi0Ury7Jx1bnH43Fbkjqdzixm3avxOxK9QTc5GN4M1fU1tjpZXl9fm1XnlaQ8z1tFUXy5vLxsNpvNSl/6apPMZrNbSVGbxFmWZVGbTGtO83UnquVrrO7JMp/Pj5IkOR0Oh7Oqa6+7SfB/G/2NGrNLzF9jkuLWvg/rPkRc+MMCocICocLCXt+j4mOVZdlS3JWc9OdabstL+otQIanelZz0/tdyhApJ9a7kpPe/liNU/GNfrxP5ZwoWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWCBUWjjb5oclkklT94Ofn50SS8jxPHh8fTz9yvOvcruuuO36Tvn4DohBoi8Az2P0AAAAASUVORK5CYII="">",0(0.0%)
2,is_correct[bool],1. True2. False,"141,207 (71.0%)57,553 (29.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABNUlEQVR4nO3bMYrDMBCF4dFiwqaRMAYfxofIYfcQuo0xVrEJabTFVumi5g1D/u8EA/kTiPFLvXcDFJKZXc3s4n0Ihj1773fvI0ZMy7Lccs6z9yEY01o7Uko/kYKbcs7ztm33UsrD+xi85zzP71rrvO/7xczixGZmVkp5rOv6630Mhly9Dxj15X0APgexQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yk9n/y3jeh+B9UT+vqbV21FpnC/gy3idrrR1m9vS+YwSDl7jCDV4SUz6oRP1lC/etRtApX8QZGwJO+aLO2BB3ysc/54B4qAsZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZMJN+SLdilchp3wRZ2xg8AKhPwpbat8GmQsWAAAAAElFTkSuQmCC"">",0(0.0%)
3,created_at[object],1. 2022-10-20 02:28:52.5163322. 2022-04-12 16:22:41.4969743. 2022-11-21 15:15:37.0490434. 2022-11-21 15:16:04.5096515. 2022-11-21 15:18:53.0870616. 2022-11-21 15:19:44.3037057. 2022-11-21 15:21:32.2037508. 2022-11-21 15:21:41.1065859. 2022-11-21 15:22:12.25843610. 2022-11-21 15:24:35.60081811. other,"1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)198,750 (100.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADLklEQVR4nO3cwY3TYBRG0ecogmFja2QpJUAPKYItvdOFNzCswoIeriVzTgX/4kpPkfxleb1eA4Xb2Q/g/yE2MmIjIzYyYiMjNjJiIyM2MmIjIzYyt2VZvi3L8nb2Q7i+28x8nZnPZz+E63NGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3MbWZ+zsyfsx/C9S3+xYiKM0pGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kDF4IeOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEZGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKBmxkREbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYy1lVkrKvIOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKJllZr7MzKezH8L13fd9/76u6/vZD+H67uu6vj+fz9/btn2c/Riu6ziOt/vMzLZtH4/H49fZD+La/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjc5/59/nH2Q/h2o7jeFv2ff/h40kKPgsnY11Fxg8EMmIjIzYyYiMjNjJiIyM2MmIjIzYyfwFj50uCzHpkKgAAAABJRU5ErkJggg=="">",0(0.0%)
4,user_id[int64],Mean (sd) : 38016.7 (15961.1)min < med < max:2315.0 < 50104.0 < 53189.0IQR (CV) : 29891.0 (2.4),"1,417 distinct values","<img src =

### Add Skill Percentile Column

In [14]:
# Convert user_level to percentile rank
# First, remove any NaN values for the calculation
valid_levels = df['user_level'].dropna()

# Calculate percentile rank for each value
# Sort values and get rank position for each score
sorted_vals = valid_levels.sort_values()
ranks = valid_levels.map(lambda x: (sorted_vals < x).sum())

# Convert to percentiles (rank / total count * 100) and round to integer
percentile_ranks = pd.Series(
    (ranks / len(valid_levels) * 100).round(),
    index=df.index
)

# Add as new column
df['user_level_percentile'] = percentile_ranks


### Extract Options
The original dataset had options in a single column, with multiple formatted options separated by newlines.
Here are some examples of the formatting:

Example 1: ["\`144π cm^3\`", "\`162π cm^3\`", "\`216π cm^3\`", "\`432π cm^3\`", "\`288π cm^3\`"]

Example 2: "---\n- \"\`10\`\"\n- \"\`1/10\`\"\n- \"\`7/10\`\"\n- \"\`-7/10\`\"\n- \"\`-1/10\`\"\n"

The use utilities.py function extract_options to parse these options into separate columns.


In [15]:
# Apply the function and create separate columns for options
df[['option_a', 'option_b', 'option_c', 'option_d', 'option_e']] = pd.DataFrame(
    df['options'].apply(extract_options).tolist(), index=df.index
)

Error parsing YAML string: while scanning for the next token
found character '`' that cannot start any token
  in "<unicode string>", line 2, column 11:
    - \"`4` : `8`\"
              ^
Error parsing YAML string: while scanning for the next token
found character '`' that cannot start any token
  in "<unicode string>", line 2, column 11:
    - \"`4` : `8`\"
              ^
Error parsing YAML string: while scanning a simple key
  in "<unicode string>", line 3, column 1:
    e 1` , `b = 3` y `c = 0`\"
    ^
could not find expected ':'
  in "<unicode string>", line 4, column 1:
    - \"`a = 1`, `b` y `c` cualquier ... 
    ^
Error parsing YAML string: while scanning a simple key
  in "<unicode string>", line 3, column 1:
    e 1` , `b = 3` y `c = 0`\"
    ^
could not find expected ':'
  in "<unicode string>", line 4, column 1:
    - \"`a = 1`, `b` y `c` cualquier ... 
    ^
Error parsing YAML string: while scanning a simple key
  in "<unicode string>", line 3, column 1:
    e 1` , `b = 

Let's check if there are any rows where the correct_option doesn't match any of the options.

In [16]:
# Create boolean masks checking if correct_option and student_answer match any of the options
correct_matches_mask = (df['correct_option'] == df['option_a']) | \
                      (df['correct_option'] == df['option_b']) | \
                      (df['correct_option'] == df['option_c']) | \
                      (df['correct_option'] == df['option_d']) | \
                      (df['correct_option'] == df['option_e'])

student_matches_mask = (df['student_answer'] == df['option_a']) | \
                      (df['student_answer'] == df['option_b']) | \
                      (df['student_answer'] == df['option_c']) | \
                      (df['student_answer'] == df['option_d']) | \
                      (df['student_answer'] == df['option_e'])

# Add columns indicating if options match
df['correct_option_matches'] = correct_matches_mask
df['student_answer_matches'] = student_matches_mask

# Count how many don't match for each
correct_non_matching = (~correct_matches_mask).sum()
student_non_matching = (~student_matches_mask).sum()
print(f"Number of rows where correct_option doesn't match any option: {correct_non_matching}")
print(f"Number of rows where student_answer doesn't match any option: {student_non_matching}")


Number of rows where correct_option doesn't match any option: 442
Number of rows where student_answer doesn't match any option: 2676


And if there are any rows where all options are null.

In [17]:
# Create mask for rows where some options are null
some_null_mask = df[['option_a', 'option_b', 'option_c', 'option_d', 'option_e']].isnull().any(axis=1)


Let's drop these rows.

In [18]:
# Drop the rows with null in some of the options or where correct_option doesn't match any option or student_answer doesn't match any option
df = df[~some_null_mask & correct_matches_mask & student_matches_mask]

print(f"Total rows left: {len(df)}")


Total rows left: 196083


Create new column that stores the letter of the correct answer. This is used to compare it with the answer of the LLM

In [19]:
# Create a new column that stores which letter (a-e) corresponds to the correct option
df['correct_option_letter'] = None

# Compare correct_option with each option column and assign corresponding letter
df.loc[df['correct_option'] == df['option_a'], 'correct_option_letter'] = 'a'
df.loc[df['correct_option'] == df['option_b'], 'correct_option_letter'] = 'b'
df.loc[df['correct_option'] == df['option_c'], 'correct_option_letter'] = 'c'
df.loc[df['correct_option'] == df['option_d'], 'correct_option_letter'] = 'd'
df.loc[df['correct_option'] == df['option_e'], 'correct_option_letter'] = 'e'


Create another column that stores the student_answer letter.

In [20]:
# Create a new column that stores which letter (a-e) corresponds to the student's answer
df['student_answer_letter'] = None

# Compare student_answer with each option column and assign corresponding letter
df.loc[df['student_answer'] == df['option_a'], 'student_answer_letter'] = 'a'
df.loc[df['student_answer'] == df['option_b'], 'student_answer_letter'] = 'b'
df.loc[df['student_answer'] == df['option_c'], 'student_answer_letter'] = 'c'
df.loc[df['student_answer'] == df['option_d'], 'student_answer_letter'] = 'd'
df.loc[df['student_answer'] == df['option_e'], 'student_answer_letter'] = 'e'


### Add age based on grade

In [22]:
# Load file in data/original/grade_age.csv
grade_age = pd.read_csv('../data/original/grade_age.csv')

# Merge df with grade_age on grade_id to add student_age column
df = df.merge(grade_age[['grade_id', 'student_age']], 
             left_on='grade_id', 
             right_on='grade_id', 
             how='left')

# Drop the redundant id column from the merge
df.drop('grade_id', axis=1, inplace=True)

In [23]:
# Save df to csv
df.to_csv('../data/new/master_with_engineered_features.csv', index=False)


In [22]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,answer_id[int64],Mean (sd) : 1461728.6 (80772.7)min < med < max:1321329.0 < 1462145.0 < 1603613.0IQR (CV) : 138126.0 (18.1),"196,083 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACvklEQVR4nO3ZsU/bQBiG8dcJQU5CbQVS2LJ0ROoQiYWlW8f+sV27dQCpM0OXTiwICZlEcYlinHRBlSpVEN8R4I2e38z5TuL5HHIkq9VKwFvXeu0DAOvYee0DbEqSJF1JuxGPWKxWq7vnOg/ibGWoSZJ0Dw4OvmRZNgh9xnQ6LZIk+RoSa+SQtCQtA9fGrn+zw7mVoUrazbJscHp6epfn+bzp4slkkp6dnQ1ubm52JTX6xcUMSV3XnbIsP+zt7f1qtVrVS6+PGU4pekAfHZInQ93k5hvcO1sul508z4vDw8Pfgdt3A9cFD8nl5eXg4uLi/Xg8/nl0dFQ03ThmfcxwSvGfYk8NyaOhxm5+e3s7S5Lkm6TGbzVJ6f7+/uc8z/tNF1ZVldZ1fTyfz79LCgq1ruuOpCxJkqZLg4ekKIquJPX7/XnIgMWuV/hwShEDus6QPPVGDd786urq3fn5+afRaDRst9uNQ62qKpV0fHJy8mM4HE6brH14s/Tu7+/bTfeVpLIsO1VVfRyNRu2mZ3+OIXGW5/lGhmStv1FDNi+KopumaW88Hs8jPsZ6aZpWoW+mUIvFYif07LFDgv/b+Jep2I+x1xRy9rdw7m3EhT8sECosECosECosbOt/phAg4u5Yerg/fuYj/UWokBR3dyxt/v6YUCEp7u5Y2vz9MaHiH2/13psvU7BAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLBAqLCws84PTSaTtOmDZ7NZKkllWabX19e9l1zvurfruWPXr9PXH9XveMyZix6SAAAAAElFTkSuQmCC"">",0(0.0%)
2,is_correct[bool],1. True2. False,"140,730 (71.8%)55,353 (28.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABO0lEQVR4nO3bMWrDQBCF4dkgTNzsIgQ6jA6Rw+YQexthtEVs3GwKF2pSSC7eMOT/TjDg3wYLvdR7N0AhmdnVzC7eh+Atz9773fuIo4Zpmr5yzqP3ITivtXZLKX1HCW7IOY/LstxLKQ/vY3Dctm2ftdZxXdeLmcWIzcyslPKY5/nH+xicdvU+4IwP7wPwfxAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CAzmL1exPM+BOdE/MyG1tqt1jpasBfx8Hot3Mye3nccxeAltlCDl8SUDyoRf9lCfZuxCzflizZfwy7UlC/ifA27iFM+/jUHxUNdyBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CATasoX5U78LdyUL9p8DTsGL5D5BQpbat+rUw8kAAAAAElFTkSuQmCC"">",0(0.0%)
3,created_at[object],1. 2022-10-20 02:28:52.5163322. 2022-04-19 16:25:07.3292123. 2022-04-12 16:26:28.8688784. 2022-04-12 16:27:26.9153505. 2022-04-12 16:28:26.9561916. 2022-04-12 16:28:39.6283317. 2022-04-12 16:35:13.7588998. 2022-04-12 16:35:49.7618669. 2022-04-19 16:13:56.77304710. 2022-04-19 16:14:22.54432011. other,"1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)196,073 (100.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADLklEQVR4nO3cwY3TYBRG0ecogmFja2QpJUAPKYItvdOFNzCswoIeriVzTgX/4kpPkfxleb1eA4Xb2Q/g/yE2MmIjIzYyYiMjNjJiIyM2MmIjIzYyt2VZvi3L8nb2Q7i+28x8nZnPZz+E63NGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3MbWZ+zsyfsx/C9S3+xYiKM0pGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kDF4IeOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEZGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKBmxkREbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYy1lVkrKvIOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKJllZr7MzKezH8L13fd9/76u6/vZD+H67uu6vj+fz9/btn2c/Riu6ziOt/vMzLZtH4/H49fZD+La/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjc5/59/nH2Q/h2o7jeFv2ff/h40kKPgsnY11Fxg8EMmIjIzYyYiMjNjJiIyM2MmIjIzYyfwFj50uCzHpkKgAAAABJRU5ErkJggg=="">",0(0.0%)
4,user_id[int64],Mean (sd) : 37926.6 (15973.8)min < med < max:2315.0 < 50099.0 < 53189.0IQR (CV) : 29836.0 (2.4),"1,

In [24]:
# Get 20 random unique students, stratified by age groups 11-18
# First get unique students with their ages
student_ages = df[['user_id', 'student_age']].drop_duplicates()

# Remove students with missing ages
student_ages = student_ages.dropna(subset=['student_age'])

# We want ~2-3 students from each age (11-18 = 8 age groups)
random_students = []
for age in range(11, 19):  # 11 to 18 inclusive
    age_students = student_ages[student_ages['student_age'] == age]['user_id'].values
    if len(age_students) >= 3:
        random_students.extend(np.random.choice(age_students, size=3, replace=False))
    else:
        random_students.extend(age_students)  # Take all if less than 3 available

# If we need more students to reach 20, randomly sample from remaining
if len(random_students) < 20:
    remaining = list(set(student_ages['user_id']) - set(random_students))
    additional = np.random.choice(remaining, size=20-len(random_students), replace=False)
    random_students.extend(additional)

# Trim to exactly 20 if we got more
random_students = random_students[:20]

# Get 10 rows that are correct and is answered by one of these students
correct_answers = df[
    (df['is_correct'] == True) & 
    (df['user_id'].isin(random_students))
].sample(n=10)

# Get 10 incorrect answers from the left over students
# Get 10 incorrect answers from the left over students
incorrect_answers = df[
    (df['is_correct'] == False) & 
    (df['user_id'].isin(random_students)) & (~df['user_id'].isin(correct_answers['user_id']))
].sample(n=10)


# Combine the samples
df_database = pd.concat([correct_answers, incorrect_answers])

# Select only the columns we need
df_database = df_database[[
    'answer_id',
    'is_correct',
    'question_title',
    'correct_option_letter',
    'student_answer_letter',
    'student_age',
    'user_level',
    'user_level_percentile',
    'topic_name',
    'subject_name',
    'axis_name',
    'option_a',
    'option_b',
    'option_c',
    'option_d',
    'option_e',
    'student_axis_correct',
    'student_topic_correct',
    'student_subject_correct',
    'student_axis_attempts',
    'student_topic_attempts',
    'student_subject_attempts'
]]

# Save to csv  
df_database.to_csv('data/new/20-test-students.csv', index=False)

## LangSmith dataset


In [31]:
from langsmith import Client

client = Client()

# Get 20 random unique students, stratified by age groups 11-18
# First get unique students with their ages
student_ages = df[['user_id', 'student_age']].drop_duplicates()

# Remove students with missing ages
student_ages = student_ages.dropna(subset=['student_age'])

# We want ~2-3 students from each age (11-18 = 8 age groups)
random_students = []
for age in range(11, 19):  # 11 to 18 inclusive
    age_students = student_ages[student_ages['student_age'] == age]['user_id'].values
    if len(age_students) >= 3:
        random_students.extend(np.random.choice(age_students, size=3, replace=False))
    else:
        random_students.extend(age_students)  # Take all if less than 3 available

# If we need more students to reach 20, randomly sample from remaining
if len(random_students) < 20:
    remaining = list(set(student_ages['user_id']) - set(random_students))
    additional = np.random.choice(remaining, size=20-len(random_students), replace=False)
    random_students.extend(additional)

# Trim to exactly 20 if we got more
random_students = random_students[:20]

# Get 10 rows that are correct and is answered by one of these students
correct_answers = df[
    (df['is_correct'] == True) & 
    (df['user_id'].isin(random_students))
].sample(n=10)

# Get 10 incorrect answers from the left over students
# Get 10 incorrect answers from the left over students
incorrect_answers = df[
    (df['is_correct'] == False) & 
    (df['user_id'].isin(random_students)) & (~df['user_id'].isin(correct_answers['user_id']))
].sample(n=10)


# Combine the samples
df_database = pd.concat([correct_answers, incorrect_answers])

# Extract the relevant columns for inputs, output, and metadata
structured_input_examples = []
for _, row in df_database.iterrows():
    # Create the input dictionary based on the input columns
    inputs = {
        "optionA": row["option_a"],
        "optionB": row["option_b"],
        "optionC": row["option_c"],
        "optionD": row["option_d"],
        "optionE": row["option_e"],
        "axisName": row["axis_name"],
        "question": row["question_title"],
        "topicName": row["topic_name"],
        "userLevel": round(row["user_level"], 2),
        "subjectName": row["subject_name"],
        "correctAnswer": row["correct_option_letter"],
        "userLevelPercentile": row["user_level_percentile"],
        "axisCorrectQuestions": row["student_axis_correct"],
        "topicCorrectQuestions": row["student_topic_correct"],
        "axisAttemptedQuestions": row["student_axis_attempts"],
        "subjectCorrectQuestions": row["student_subject_correct"],
        "topicAttemptedQuestions": row["student_topic_attempts"],
        "subjectAttemptedQuestions": row["student_subject_attempts"],
        "studentAge": row["student_age"]
    }
    # The output is the student_answer_letter column
    output = row["student_answer_letter"]
    
    # Extract metadata - all columns except the input columns
    metadata = row.drop(["option_a", "option_b", "option_c", "option_d", "option_e", "axis_name", "question_title", "topic_name", "user_level", "subject_name", "correct_option_letter", "user_level_percentile", "student_axis_correct", "student_topic_correct", "student_axis_attempts", "student_subject_correct", "student_topic_attempts", "student_subject_attempts", "student_age"]).to_dict()
    
    # Append to the examples list, include metadata only if it exists
    if metadata:
        structured_input_examples.append((inputs, output, metadata))
    else:
        structured_input_examples.append((inputs, output))

# Assume we have a client API similar to the example provided
structured_input_dataset_name = "20-test-students"
if not client.has_dataset(dataset_name=structured_input_dataset_name):
    structured_input_dataset = client.create_dataset(
        dataset_name=structured_input_dataset_name
    )
    for input_tuple in structured_input_examples:
        metadata = None
        if len(input_tuple) == 3:
            inputs, answer, metadata = input_tuple
        else:
            inputs, answer = input_tuple
        client.create_example(
            inputs=inputs,
            outputs={"output": answer},
            dataset_id=structured_input_dataset.id,
            metadata=metadata,
        )

## Langchain Experiments

### Baseline
We use a [simple prompt](https://smith.langchain.com/prompts/baseline-prompt?organizationId=25e81f1f-5303-4e8e-bca2-1f59955be6df) that asks the LLM to simulate a student, without any additional context.

In [6]:
def base_experiment(inputs):
    prompt_template = hub.pull("baseline-prompt")

    parser = JsonOutputParser()

    json_llm = gpt4o_mini.bind(response_format={"type": "json_object"})

    chain = prompt_template | json_llm | parser
    result = chain.invoke({
        "topicName": inputs["topicName"],
        "subjectName": inputs["subjectName"],
        "axisName": inputs["axisName"],
        "question": inputs["question"],
        "optionA": inputs["optionA"],
        "optionB": inputs["optionB"],
        "optionC": inputs["optionC"],
        "optionD": inputs["optionD"],
        "optionE": inputs["optionE"],
        "studentAge": inputs["studentAge"]
    })

    return {"output": result}

base_experiment_results = evaluate(
    base_experiment,
    data="20-test-students",
    experiment_prefix="base_experiment",
    evaluators=evaluators,
    num_repetitions=1
)

/Users/matiashoyl/miniconda3/envs/capstone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'base_experiment-b9011f6f' at:
https://smith.langchain.com/o/25e81f1f-5303-4e8e-bca2-1f59955be6df/datasets/65c90f29-e6fb-4227-9b79-2bbe38a0c780/compare?selectedSessions=86d24032-bf21-4cb2-844c-a5d4d16a576a




0it [00:00, ?it/s]/Users/matiashoyl/miniconda3/envs/capstone/lib/python3.11/site-packages/langsmith/client.py:5574: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))
20it [00:08,  2.25it/s]


### Experiment 1: Percentile

In [ ]:
def percentile_experiment(inputs):
    prompt_template = hub.pull("student-simulator-percentile")

    parser = JsonOutputParser()

    json_llm = gpt4o_mini.bind(response_format={"type": "json_object"})

    chain = prompt_template | json_llm | parser
    result = chain.invoke({
        "userLevelPercentile": inputs["userLevelPercentile"],
        "topicName": inputs["topicName"],
        "topicAttemptedQuestions": inputs["topicAttemptedQuestions"],
        "topicCorrectQuestions": inputs["topicCorrectQuestions"],
        "subjectName": inputs["subjectName"],
        "subjectAttemptedQuestions": inputs["subjectAttemptedQuestions"],
        "subjectCorrectQuestions": inputs["subjectCorrectQuestions"],
        "axisName": inputs["axisName"],
        "axisAttemptedQuestions": inputs["axisAttemptedQuestions"],
        "axisCorrectQuestions": inputs["axisCorrectQuestions"],
        "question": inputs["question"],
        "optionA": inputs["optionA"],
        "optionB": inputs["optionB"],
        "optionC": inputs["optionC"],
        "optionD": inputs["optionD"],
        "optionE": inputs["optionE"],
        "studentAge": inputs["studentAge"]
    })

    return {"output": result}

percentile_experiment_results = evaluate(
    percentile_experiment,
    data="20-test-students",
    experiment_prefix="percentile_experiment",
    evaluators=evaluators,
    num_repetitions=10
)

### Experiment 2: Raw Level

In [6]:
def raw_level_experiment(inputs):
    prompt_template = hub.pull("student-simulator-raw-level")

    parser = JsonOutputParser()

    json_llm = gpt4o_mini.bind(response_format={"type": "json_object"})

    chain = prompt_template | json_llm | parser
    result = chain.invoke({
        "userLevelPercentile": inputs["userLevelPercentile"],
        "topicName": inputs["topicName"],
        "topicAttemptedQuestions": inputs["topicAttemptedQuestions"],
        "topicCorrectQuestions": inputs["topicCorrectQuestions"],
        "subjectName": inputs["subjectName"],
        "subjectAttemptedQuestions": inputs["subjectAttemptedQuestions"],
        "subjectCorrectQuestions": inputs["subjectCorrectQuestions"],
        "axisName": inputs["axisName"],
        "axisAttemptedQuestions": inputs["axisAttemptedQuestions"],
        "axisCorrectQuestions": inputs["axisCorrectQuestions"],
        "question": inputs["question"],
        "optionA": inputs["optionA"],
        "optionB": inputs["optionB"],
        "optionC": inputs["optionC"],
        "optionD": inputs["optionD"],
        "optionE": inputs["optionE"],
        "studentAge": inputs["studentAge"],
        "userLevel": inputs["userLevel"]
    })

    return {"output": result}

percentile_experiment_results = evaluate(
    raw_level_experiment,
    data="20-test-students",
    experiment_prefix="raw_level_experiment",
    evaluators=evaluators,
    num_repetitions=1
)

/Users/matiashoyl/miniconda3/envs/capstone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'raw_level_experiment-cafdba31' at:
https://smith.langchain.com/o/25e81f1f-5303-4e8e-bca2-1f59955be6df/datasets/65c90f29-e6fb-4227-9b79-2bbe38a0c780/compare?selectedSessions=eb589c11-228a-42c8-a9ed-aa27742c72b7




0it [00:00, ?it/s]/Users/matiashoyl/miniconda3/envs/capstone/lib/python3.11/site-packages/langsmith/client.py:5574: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))
20it [00:08,  2.23it/s]


### Experiment 3: Rubric Level

In [6]:
def rubric_level_experiment(inputs):
    prompt_template = hub.pull("student-simulator-rubric-level")

    parser = JsonOutputParser()

    # json_llm = o1_mini.bind(response_format={"type": "json_object"})

    chain = prompt_template | o1_mini | parser
    result = chain.invoke({
        "userLevelPercentile": inputs["userLevelPercentile"],
        "topicName": inputs["topicName"],
        "topicAttemptedQuestions": inputs["topicAttemptedQuestions"],
        "topicCorrectQuestions": inputs["topicCorrectQuestions"],
        "subjectName": inputs["subjectName"],
        "subjectAttemptedQuestions": inputs["subjectAttemptedQuestions"],
        "subjectCorrectQuestions": inputs["subjectCorrectQuestions"],
        "axisName": inputs["axisName"],
        "axisAttemptedQuestions": inputs["axisAttemptedQuestions"],
        "axisCorrectQuestions": inputs["axisCorrectQuestions"],
        "question": inputs["question"],
        "optionA": inputs["optionA"],
        "optionB": inputs["optionB"],
        "optionC": inputs["optionC"],
        "optionD": inputs["optionD"],
        "optionE": inputs["optionE"],
        "studentAge": inputs["studentAge"],
        "userLevel": inputs["userLevel"]
    })

    return {"output": result}

rubric_level_experiment_results = evaluate(
    rubric_level_experiment,
    data="20-test-students",
    experiment_prefix="rubric_level_experiment",
    evaluators=evaluators,
    num_repetitions=1
)

/Users/matiashoyl/miniconda3/envs/capstone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rubric_level_experiment-a6d8da00' at:
https://smith.langchain.com/o/25e81f1f-5303-4e8e-bca2-1f59955be6df/datasets/65c90f29-e6fb-4227-9b79-2bbe38a0c780/compare?selectedSessions=02bbd936-232e-4b47-8b7c-8eb06a6852e3




0it [00:00, ?it/s]/Users/matiashoyl/miniconda3/envs/capstone/lib/python3.11/site-packages/langsmith/client.py:5574: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))
20it [00:22,  1.12s/it]


Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=a2012262-3a61-4ad4-90bc-01750b62ba51,id=a2012262-3a61-4ad4-90bc-01750b62ba51; trace=2b37cd0d-ccbd-48b8-b216-73ce90ed2bd3,id=2b37cd0d-ccbd-48b8-b216-73ce90ed2bd3; trace=3848c702-afe5-45d1-b213-64d371f461ce,id=3848c702-afe5-45d1-b213-64d371f461ce; trace=7f91551a-7c95-4811-8145-f6f472ff696f,id=7f91551a-7c95-4811-8145-f6f472ff696f; trace=478c63d0-2ec9-4588-9177-038ca781e0a8,id=478c63d0-2ec9-4588-9177-038ca781e0a8; trace=560e478a-2aad-4330-abb4-84781c6fcbf7,id=560e478a-2aad-4330-abb4-84781c6fcbf7; trace=358db774-57f8-4e65-8169-cfa629fc8f26,id=358db774-57f8-4e65-8169-cfa629fc8f26; trace=f3d02bdd-1644-40d5-802d-1aa484c77679,id=f3d02bdd-1644-40d5-802d-1aa484c77679; trace=3d0b5644-f41f

## Analogue Experiments

### Baseline

In [8]:
def process_single_row(row, prompt_generator, model_caller, response_parser):
    """Process a single row of test data through the LLM pipeline.
    
    Args:
        row (pd.Series): Row of test data
        prompt_generator (callable): Function to generate prompt from row data
        model_caller (callable): Function to call LLM model
        response_parser (callable): Function to parse LLM response
    
    Returns:
        dict: Results dictionary or None if error
    """
    # Generate prompt
    prompt = prompt_generator(
        student_age=row['student_age'],
        topic_name=row['topic_name'],
        subject_name=row['subject_name'],
        axis_name=row['axis_name'],
        question=row['question_title'],
        option_a=row['option_a'],
        option_b=row['option_b'],
        option_c=row['option_c'],
        option_d=row['option_d'],
        option_e=row['option_e']
    )

    # Get model response
    response = model_caller(prompt)
    if not response:
        return None

    # Parse response
    parsed_response = response_parser(response)
    if not parsed_response:
        return None

    # Calculate metrics
    metrics = calculate_metrics(
        llm_answer=parsed_response['answer'],
        correct_answer=row['correct_option_letter'],
        student_answer=row['student_answer_letter']
    )

    # Combine results
    return {
        **row,
        **parsed_response,
        **metrics
    }

def calculate_metrics(llm_answer, correct_answer, student_answer):
    """Calculate evaluation metrics for a single response.
    
    Args:
        llm_answer (str): LLM's answer
        correct_answer (str): Correct answer
        student_answer (str): Student's answer
        
    Returns:
        dict: Dictionary of metrics
    """
    is_llm_correct = llm_answer == correct_answer
    is_student_correct = student_answer == correct_answer
    
    return {
        "is_llm_correct": is_llm_correct,
        "llm_match_student_answer": llm_answer == student_answer,
        "llm_match_student_correct": is_llm_correct == is_student_correct
    }

def run_experiment(
    df_test, 
    prompt_generator, 
    model_caller, 
    response_parser,
    output_path,
    experiment_name="experiment"
):
    """Run complete experiment pipeline.
    
    Args:
        df_test (pd.DataFrame): Test dataset
        prompt_generator (callable): Function to generate prompts
        model_caller (callable): Function to call LLM
        response_parser (callable): Function to parse responses
        output_path (str): Path to save results
        experiment_name (str): Name of experiment for logging
    """
    results = []
    total_rows = len(df_test)

    for idx, row in df_test.iterrows():
        result = process_single_row(row, prompt_generator, model_caller, response_parser)
        
        if result:
            results.append(result)
            log_progress(idx, total_rows, result, experiment_name)
        else:
            print(f"Error processing row {idx} in {experiment_name}")

    # Save results
    if results:
        df_results = pd.DataFrame(results)
        df_results.to_csv(output_path, index=False)
        print(f"\nResults saved to {output_path}")
    
    return results

def log_progress(idx, total_rows, result, experiment_name):
    """Log progress and results for a single row.
    
    Args:
        idx (int): Current row index
        total_rows (int): Total number of rows
        result (dict): Results dictionary
        experiment_name (str): Name of experiment
    """
    print(
        f"[{experiment_name}] Row {idx}/{total_rows} | "
        f"LLM Answer: {result.get('answer')} | "
        f"LLM Correct: {result.get('is_llm_correct')} | "
        f"Matches Student Answer: {result.get('llm_match_student_answer')} | "
        f"Matches Student Correctness: {result.get('llm_match_student_correct')}"
    )

# Usage example:
if __name__ == "__main__":
    # Load data
df_test = pd.read_csv('data/new/20-test-students.csv')

Row 0/20 | LLM Answer: e | LLM Correct: True | Matches Student Answer: True | Matches Student Correctness: True
Row 1/20 | LLM Answer: d | LLM Correct: True | Matches Student Answer: True | Matches Student Correctness: True
Row 2/20 | LLM Answer: e | LLM Correct: True | Matches Student Answer: True | Matches Student Correctness: True
Row 3/20 | LLM Answer: a | LLM Correct: False | Matches Student Answer: False | Matches Student Correctness: False
Row 4/20 | LLM Answer: e | LLM Correct: True | Matches Student Answer: True | Matches Student Correctness: True
Row 5/20 | LLM Answer: e | LLM Correct: True | Matches Student Answer: True | Matches Student Correctness: True
Row 6/20 | LLM Answer: b | LLM Correct: True | Matches Student Answer: True | Matches Student Correctness: True
Row 7/20 | LLM Answer: d | LLM Correct: False | Matches Student Answer: False | Matches Student Correctness: False
Row 8/20 | LLM Answer: d | LLM Correct: True | Matches Student Answer: True | Matches Student Corr